In [1]:
import argparse
import os
import math
import random
import json
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from encoder import Encoder
from attention import Attention
from decoder import Decoder
from generator import Generator

from data_loader import SpeechDataset, Padding, ToTensor

Using TensorFlow backend.


In [35]:
with open('labels_dict.json', 'r') as f:
    labels = json.loads(f.read())
    
id2label = {v: k for k, v in labels.items()}

In [83]:
audio_conf = {'window': 'hamming',
              'window_size' : 0.02,
              'window_stride' : 0.01,
              'sampling_rate': 16000}

val_dataset = SpeechDataset('../../SpeechRecognition.EN/deepspeech.cv/data/cv-other-dev_manifest.csv', 
                            'labels_dict.json',
                            audio_conf,
                            transform=transforms.Compose([Padding(SIGNAL_INPUT_SIZE, TXT_INPUT_SIZE, 'labels_dict.json')]) 
                              )

In [84]:
torch.cuda.set_device(1)

In [85]:
SIGNAL_INPUT_SIZE = 1500 
TXT_INPUT_SIZE = 135
OUTPUT_DIM = 33
BATCH_SIZE = 32

NUM_GRU = 6
ENC_HID_DIM = 256
DEC_HID_DIM = 256 
DEC_EMB_DIM = 256
DROPOUT_RATE = 0.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

encoder = Encoder(SIGNAL_INPUT_SIZE, NUM_GRU, ENC_HID_DIM, DEC_HID_DIM, DROPOUT_RATE, device)
attention = Attention(enc_hid_dim=ENC_HID_DIM, dec_hid_dim=DEC_HID_DIM)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DROPOUT_RATE, attention)
model = Generator(encoder, decoder, device).to(device)
model = model.eval()

In [86]:
signal, txt_ids = torch.from_numpy(val_dataset[1]['signal']), torch.from_numpy(val_dataset[1]['transcript'])
signal = signal.type(torch.FloatTensor).to(device)
txt_ids = txt_ids.type(torch.LongTensor).to(device)

In [87]:
signal.size(), txt_ids.size()

(torch.Size([161, 1500]), torch.Size([1, 135]))

In [88]:
model.load_state_dict(torch.load('models/rsr_gan.pt'))

In [89]:
_, _, output = model(signal.view(1, 161, 1500), txt_ids)

In [90]:
output.size()

torch.Size([1, 135, 33])

In [91]:
output.max(2)[1].size()

torch.Size([1, 135])

In [92]:
out_txt = ''
for l in output.max(2)[1][0].cpu().numpy():
    out_txt += id2label[l] 
    
out_txt

' iutt  e an  he  ou   o  e  ee  en   oree eeee  e  oe  ee e  on  one  e ee  oneee e ee ee eeeee  on  eee  ee e  ee e e  e e  e e  e ee '

In [93]:
true_txt = ''
for l in txt_ids[0].cpu().numpy():
    true_txt += id2label[l]
    
true_txt

'<sos>bravery is the ability to overcome fear<eos>padpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpadpad'